# Aim
To achieve 00->01, feed opt param of one opt as initial param for another

In [1]:
import sys 
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram')
#sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram\\Double-ECD\\May\\DECD')



In [2]:
#from DECD_block_fix import BatchOptimizer as BO1
from DECD import BatchOptimizer as BO

In [3]:
import numpy as np
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  # supress warnings
import h5py

# print(
#     "\nNeed tf version 2.3.0 or later. Using tensorflow version: "
#     + tf.__version__
#     + "\n"
# )
import ECD_control.ECD_optimization.tf_quantum as tfq
from ECD_control.ECD_optimization.visualization import VisualizationMixin
import qutip as qt
import datetime
import time


# Code

In [4]:
#The target oscillator state.
N1 =10
N2 =10
Fock1 = 0
Fock2= 0
psi_i1 = qt.basis(N1,Fock1) #target state
psi_i2 = qt.basis(N2,Fock2)
psi_initial = qt.tensor(psi_i1, psi_i2)

In [5]:
psi_initial

Quantum object: dims = [[10, 10], [1, 1]], shape = (100, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]

In [6]:
#The target oscillator state.
N1 =10
N2 =10
Fock1 = 1
Fock2= 0
psi_t1 = qt.basis(N1,Fock1) #target state
psi_t2 = qt.basis(N2,Fock2)
psi_target = qt.tensor(psi_t1, psi_t2)
psi_target

Quantum object: dims = [[10, 10], [1, 1]], shape = (100, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]

In [7]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
#the optimization options
opt_params = {
'N_blocks' : 5, #circuit depth
'N_multistart' : 2, #Batch size (number of circuit optimizations to run in parallel)
'epochs' : 1000, #number of epochs before termination
'epoch_size' : 20, #number of adam steps per epoch
'learning_rate' : 0.01, #adam learning rate
'term_fid' : 0.995, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'gamma_scale' : 3.0, #maximum |gamma| for random initialization
'N_cav1': N1, #number of levels in mode 1
'N_cav2': N2, #number of levels in mode 2
'initial_states' : [qt.tensor(qt.basis(2,0),psi_initial)], #qubit tensor oscillator, start in |g> |0>
'target_states' : [qt.tensor(qt.basis(2,0), psi_target)], #end in |e> |target>.
'name' : 'Fock1 %d' % Fock1, #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
'initial_params': None,
}


#note: optimizer includes pi pulse in every ECD step. However, final ECD step is implemented 
#in experiment as a displacement since the qubit and oscillator should be disentangled at this point.
#So, we ask the optimizer to end in |e> |target> instead of |g>|target>.

In [8]:
tf.config.run_functions_eagerly(True)

In [9]:
opt1 = BO(**opt_params)
opt1.print_info()

None
optimization_type: state transfer
N_multistart: 2
N_blocks: 5
term_fid: 0.995
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.01
epoch_size: 20
epochs: 1000
beta_scale: 3.0
gamma_scale: 3.0
alpha1_scale: 1.0
alpha2_scale: 1.0
theta_scale: 3.141592653589793
use_etas: False
use_displacements: False
use_phase: False
name: Fock1 1
comment: 
initial_params: None
N_cav1: 10
N_cav2: 10
filename: Fock1 1.h5

Best circuit parameters found:
betas:         [ 1.76428+1.48211j -0.66278-0.59697j -2.10909-1.70753j -0.05466+0.56143j
 -2.23997+1.32183j]
gammas:         [-0.07271+0.02716j  0.77525+0.30081j -2.20625+1.49299j  0.10692-0.45737j
  1.27751+1.32007j]
alphas1:        [0.+0.j]
alphas2:        [0.+0.j]
phis (deg):    [   0.      -171.71034 -110.26992    4.24727  -88.42216]
etas (deg):    [89.99999 89.99999 89.99999 89.99999 89.99999]
thetas (deg):  [138.6047  -83.90494 -16.15525 147.16382  52.0224 ]
Max Fidelity:  0.011080




In [10]:
#run optimizer.
opt1.optimize()

Start time: 2022-05-17 09:13:36


TypeError: Object dtype dtype('O') has no native HDF5 equivalent

In [ ]:
opt_params['learning_rate'] = 0.1
opt = BO(**opt_params)
opt.print_info()

In [ ]:
opt.optimize()

In [ ]:
opt.learning_rate = 0.01
opt.optimize()

In [ ]:
opt.dfid_stop = -10
opt.optimize()

In [ ]:
opt2 = BO(**opt_params)

# Varying Layers

In [ ]:
def compute_error(data):
    '''
    Input: List of numbers
    Output: 1/2 size of error bar
    '''
    m = np.mean(data)
    tot_diff_sq = 0
    for i in data: 
        tot_diff_sq += (i-m)**2
    mse = np.sqrt(tot_diff_sq/len(data))
    return mse/len(data)

In [ ]:
# diff number of depths
n_depths = [5*i for i in range(1,20)]
#fids for each depth
fids = []
#tested depths (for graphing purposes)
t_depths = []

In [ ]:
import time
import pylab as pl
from IPython import display
import matplotlib.pyplot as plt

errors = []
fid_arr = []
fid_plot = []
t_depths = []
fig = plt.figure()

for n in n_depths: 
    print('Number of depth/ DECD blocks: ' + str(n))
    
    
    #update tested depths 
    t_depths.append(n)
    #change opt_params
    opt_params['N_blocks'] = n
    
    #Do So 100 times
    N_times = 50
    fid_layers = []
    
    for i in range(N_times):
        #display.clear_output(wait=True)
        #create new optimizer 
        opt = BO2(**opt_params)
        #run optimizer.
        opt.optimize()
        #collect fidelity
        fid = opt.best_fidelity()
        #store it  
        fid_layers.append(fid)
    
    err = compute_error(fid_layers)
    errors.append(err)
    fid_plot.append(np.mean(fid_layers))
    fid_arr.append(fid_layers)
    
    #we prepare plot to track this stuff live--------
    plt.errorbar(t_depths, fid_plot, yerr = errors, linestyle = '-', marker = 'o')

    plt.xlabel('Circuit Depth/ No. of DECD blocks')
    plt.ylabel('Optimized Fidelity')
    display.clear_output(wait=True)
    display.display(fig)
    #--------------------------------

In [ ]:
errors

In [ ]:
fid_arr

In [ ]:
import numpy as np
data = [errors, fid_arr]
fname = 'Test_00_to_10_Fids_Errorbars.txt'
a_file = open(fname, "w")
np.savetxt(a_file, data)
a_file.close()